# GPT-2

## Load Data and Tokenize

In [1]:
# Shakespear dataset
data_path = "../../data/input.txt"

with open(data_path, "r", encoding="utf-8") as file:
    text = file.read()

In [2]:
data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
import tiktoken

In [5]:
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode(data)
print(tokens[:24])


[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


## Get batches for training

In [6]:
import torch

In [11]:
buf = torch.tensor(tokens[:24 + 1]) # +1 for the next token target

In [12]:
buf

tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13,   198])

In [14]:
# Reshape to get 4 batches of 6 tokens
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
print(x)
print(y)


tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])
